In [1]:
import pandas as pd
import re 

In [2]:
df = pd.read_csv('/Users/Howard/Desktop/housing.csv',skiprows=1)
# df = pd.read_csv('C:/Users/student/Desktop/housing.csv',skiprows=1)
df

/Users/bay/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,The villages and towns urban district,transaction sign,land sector position building sector house number plate,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,transaction pen number,shifting level,...,building present situation pattern - health,building present situation pattern - compartmented,Whether there is manages the organization,total price NTD,the unit price (NTD / square meter),the berth category,berth shifting total area square meter,the berth total price NTD,the note,serial number
0,文山區,房地(土地+建物)+車位,台北市文山區興隆路四段145巷1~30號,16.40,住,NaN,NaN,1011012,土地1建物1車位1,七層,...,1,有,有,13550000,146147.0,坡道機械,17.45,0,NaN,RPQNMLMJJIHFFAA98CA
1,文山區,房地(土地+建物),台北市文山區指南路三段22巷8弄1~30號,52.66,住,NaN,NaN,1011009,土地1建物1車位0,六層，電梯樓梯間,...,2,有,有,17700000,108010.0,NaN,0.00,0,1.價格含增建2.其中一衛為0.5衛浴,RPSNMLMJJIHFFAA56CA
2,文山區,土地,興隆段四小段241~270地號,1.04,其他,NaN,NaN,1011026,土地3建物0車位0,NaN,...,0,有,無,100397,96166.0,NaN,0.00,0,NaN,RPQQMLMJJIHFFAA66CA
3,文山區,房地(土地+建物),台北市文山區文山區木新路三段50巷7弄1~30號,23.67,住,NaN,NaN,1011011,土地1建物1車位0,一層，平台,...,1,有,有,9900000,177419.0,NaN,0.00,0,NaN,RPQNMLNJJIHFFAA09CA
4,文山區,土地,興泰段二小段121~150地號,53.00,其他,NaN,NaN,1011019,土地1建物0車位0,NaN,...,0,有,無,4160500,78500.0,NaN,0.00,0,管理機關:臺北市政府工務局公園路燈工程管理處.協議價購,RPXNMLRJJIHFFAA57CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629081,新店區,房地(土地+建物)+車位,斯馨段31~60地號,19.70,住,NaN,NaN,1081107,土地1建物1車位1,十七層,...,2,有,無,17340000,164737.0,坡道平面,33.06,2400000,NaN,RPPTMLQJNHPFFCF56CB
629082,新店區,房地(土地+建物)+車位,斯馨段31~60地號,12.25,住,NaN,NaN,1081020,土地1建物1車位1,五層,...,1,有,無,11500000,161778.0,坡道平面,33.06,2400000,NaN,RPQTMLQJNHPFFCF66CB
629083,新店區,房地(土地+建物)+車位,斯馨段31~60地號,12.25,住,NaN,NaN,1090106,土地1建物1車位1,三層,...,1,有,無,11520000,162133.0,坡道平面,33.06,2400000,NaN,RPRTMLQJNHPFFCF76CB
629084,新店區,房地(土地+建物)+車位,斯馨段31~60地號,19.70,住,NaN,NaN,1080628,土地1建物1車位1,九層,...,2,有,無,16310000,154482.0,坡道平面,33.06,2300000,NaN,RPTTMLQJNHPFFCF96CB


# 刪除transaction sign為土地、車位的資料列

In [3]:
mask = df["transaction sign"].isin(["土地","車位"])
taipei = df[-mask]

# 刪除transaction sign、the berth category、the note、serial number欄位

In [4]:
taipei = taipei.drop(["transaction sign","the berth category","the note","serial number"],axis=1)

# 拆分transaction pen number欄位

In [5]:
taipei['transaction pen number'] = taipei['transaction pen number'].str.replace('土地',',')
taipei['transaction pen number'] = taipei['transaction pen number'].str.replace('建物',',')
taipei['transaction pen number'] = taipei['transaction pen number'].str.replace('車位',',')

taipei['land'] = taipei['transaction pen number'].str.split(',').str.get(1).astype('int64')
taipei['building'] = taipei['transaction pen number'].str.split(',').str.get(2).astype('int64')
taipei['berth'] = taipei['transaction pen number'].str.split(',').str.get(3).astype('int64')

taipei = taipei.drop(['transaction pen number'],axis=1)

# building present situation pattern - compartmented欄位轉換為數值

In [6]:
taipei['building present situation pattern - compartmented'] = taipei['building present situation pattern - compartmented'].replace(['無','有'],[0,1]).astype('int64')

# Whether there is manages the organization欄位轉換為數值

In [7]:
taipei['Whether there is manages the organization'] = taipei['Whether there is manages the organization'].replace(['無','有'],[0,1]).astype('int64')

# transaction year month and day欄位轉換成日期格式

In [8]:
transaction_date = taipei['transaction year month and day'].to_list()

transaction_error = []
for i in transaction_date :
    if len(str(i)) < 6:
#         print(i)
        transaction_error.append(i)
    elif int(str(i)[-4:-2]) < 1 :
        transaction_error.append(i)
# transaction_error

mask2 = taipei['transaction year month and day'].isin(transaction_error)
taipei = taipei[-mask2]
# taipei

taipei["transaction year month and day"] = taipei["transaction year month and day"].astype('int64')+19110000

taipei[['transaction year month and day']] = taipei[['transaction year month and day']].applymap(str).applymap(lambda s: "{}-{}-{}".format(s[0:4],s[4:6],s[6:]))

taipei['transaction year month and day'] = pd.to_datetime(taipei['transaction year month and day'],format='%Y-%m-%d') 

# 重置index
taipei = taipei.reset_index(drop=True)

# construction to complete the years欄位轉換成日期格式

In [9]:
construction_date = taipei['construction to complete the years'].fillna(0).to_list()

construction_date_list=[]

for j in construction_date:
    a = str(j).split(".")[0]
    num = ''.join([x for x in a if x.isdigit()])
    construction_date_list.append(num) 

taipei["construction to complete the years"] = pd.Series(construction_date_list)


construction_date = taipei['construction to complete the years'].to_list()

construction_error = []
for i in construction_date :
    if len(str(i)) < 6:
        construction_error.append(i)
    elif int(str(i)[-4:-2]) < 1 :
        construction_error.append(i)
    elif str(i)[-4:-2] == '06' :
        if int(str(i)[-2:]) > 30:
            construction_error.append(i)
    elif str(i)[-4:-2] == '02' :
        if int(str(i)[-2:]) > 29:
            construction_error.append(i)
            

mask3 = taipei['construction to complete the years'].isin(construction_error)
taipei = taipei[-mask3]
# taipei

taipei["construction to complete the years"] = taipei["construction to complete the years"].astype('int64')+19110000

taipei[['construction to complete the years']] = taipei[['construction to complete the years']].applymap(str).applymap(lambda s: "{}-{}-{}".format(s[0:4],s[4:6],s[6:]))

taipei['construction to complete the years'] = pd.to_datetime(taipei['construction to complete the years'],format='%Y-%m-%d') 

# 重置index
taipei = taipei.reset_index(drop=True)

# 建立屋齡house age欄位

In [10]:
taipei['house age'] = taipei['transaction year month and day'] - taipei['construction to complete the years']

# taipei[['transaction year month and day','construction to complete the years','house age']]

# 地址_門牌號碼範圍取中間數(e.g. 門牌號碼:1~30 > 15)

In [11]:
addr_list = taipei['land sector position building sector house number plate'].to_list()

n=0 #計次用變數
new_addr_list = [] #用來存放更改過後的地址
for i in addr_list:
    try:
        rang = re.search(r"(\d+~\d+)", i)[1]
        split_rang = rang.split('~')
        number1 = int(split_rang[0])
        number2 = int(split_rang[1])
        even = ((number2-number1)//2)+number1
        new_i = i.replace(f'{rang}',f'{even}')
        new_addr_list.append(new_i)
        n+=1
#         print(n)
#         print(new_i)
    except:                    #有些地址沒有(\d+~\d+)的格式,搜尋不到會出錯,所以用except接
        new_addr_list.append(i)
        n+=1
#         print(n)
#         print(new_i)
# len(new_addr_list) #檢查新list地址數量是否和舊的list數量一致
taipei['new addr'] = pd.Series(new_addr_list)

taipei[['land sector position building sector house number plate','new addr']]

,land sector position building sector house number plate,new addr
0,台北市文山區興隆路四段145巷1~30號,台北市文山區興隆路四段145巷15號
1,台北市文山區指南路三段22巷8弄1~30號,台北市文山區指南路三段22巷8弄15號
2,台北市文山區興隆路二段233巷5弄1~30號,台北市文山區興隆路二段233巷5弄15號
3,台北市文山區羅斯福路五段218巷38弄1~30號,台北市文山區羅斯福路五段218巷38弄15號
4,台北市文山區萬盛街121~150號,台北市文山區萬盛街135號
...,...,...
466833,新北市土城區金城路二段121~150號,新北市土城區金城路二段135號
466834,新北市土城區國際路158巷1~30號,新北市土城區國際路158巷15號
466835,新北市土城區金城路二段121~150號,新北市土城區金城路二段135號
466836,新北市土城區金城路二段121~150號,新北市土城區金城路二段135號


# 總層數欄位數字化(float64)

In [12]:
#取代用變數
num1 = ['一','二','三','四','五','六','七','八','九']
rnum1 = []
for i in range(1,10):
    l = str(i)
    rnum1.append(l)
num2 = ['2十層','3十層','4十層','5十層','6十層','7十層','8十層','9十層']
rnum2 = []
for i in range(2,10):
    l = str(i)+'0層'
    rnum2.append(l)
num3 = ['2十','3十','4十','5十','6十','7十','8十','9十']
rnum3 = []
for i in range(2,10):
    l = str(i)
    rnum3.append(l)

num4 = '十層'
rnum4 = '10層'    
    
num5 = '十'
rnum5 = '1'

num6 = '地下層'
rnum6 = '-1層'

num7 = '地下'
rnum7 = '-'

num8 = '層'
rnum8 = ''

In [13]:
#帶入變數做置換
taipei['total floor number'] = taipei['total floor number'].replace(num1,rnum1,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num2,rnum2,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num3,rnum3,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num4,rnum4,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num5,rnum5,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num6,rnum6,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num7,rnum7,regex=True)
taipei['total floor number'] = taipei['total floor number'].replace(num8,rnum8,regex=True)

In [14]:
#轉換成float64
taipei['total floor number'] = taipei['total floor number'].astype('float')

In [15]:
#最終確認
taipei['total floor number'] = taipei['total floor number'].fillna(0).astype('int64')

# 建物型態數字化

In [16]:
taipei['building state'] 

0         住宅大樓(11層含以上有電梯)
1           華廈(10層含以下有電梯)
2           華廈(10層含以下有電梯)
3              套房(1房1廳1衛)
4           華廈(10層含以下有電梯)
               ...       
466833    住宅大樓(11層含以上有電梯)
466834                透天厝
466835    住宅大樓(11層含以上有電梯)
466836    住宅大樓(11層含以上有電梯)
466837    住宅大樓(11層含以上有電梯)
Name: building state, Length: 466838, dtype: object

In [17]:
bs = list(taipei['building state'].unique())
bs

['住宅大樓(11層含以上有電梯)',
 '華廈(10層含以下有電梯)',
 '套房(1房1廳1衛)',
 '公寓(5樓含以下無電梯)',
 '辦公商業大樓',
 '廠辦',
 '店面(店鋪)',
 '其他',
 '透天厝',
 '工廠',
 '倉庫',
 '農舍']

In [18]:
bname = bs
dgbname = []
for i in range(1,13):
#     x = i
    dgbname.append(i)
dgbname
bname

['住宅大樓(11層含以上有電梯)',
 '華廈(10層含以下有電梯)',
 '套房(1房1廳1衛)',
 '公寓(5樓含以下無電梯)',
 '辦公商業大樓',
 '廠辦',
 '店面(店鋪)',
 '其他',
 '透天厝',
 '工廠',
 '倉庫',
 '農舍']

In [19]:
taipei['building state'] = taipei['building state'].replace(bname,dgbname).astype('int64')

# 主要用途數字化(float64)

In [20]:
taipei['main use']

0             住家用
1             住家用
2             住家用
3             住家用
4             住家用
           ...   
466833    見其他登記事項
466834        住家用
466835    見其他登記事項
466836    見其他登記事項
466837    見其他登記事項
Name: main use, Length: 466838, dtype: object

In [21]:
len(taipei['main use'].unique())

15

In [22]:
pname = ['見其他登記事項', '見使用執照', '其他', '共有部份', '共有部分', '住家用','國民住宅', '商業用', '住商用', '工商用', '住工用','工業用', '市場攤位', '農舍', '農業用', '停車空間']
dgpname = [0, 0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [23]:
taipei['main use'] = taipei['main use'].replace(pname,dgpname)

In [24]:
taipei['main use'].unique().dtype

dtype('float64')

In [25]:
taipei['main use'] = taipei['main use'].fillna(0).astype('int64')

# 都市土地使用分區數值化

In [26]:
taipei['the use zoning or compiles and checks'] = taipei['the use zoning or compiles and checks'].replace(['住','商','工','農','其他'],[1,2,3,4,5])
taipei['the use zoning or compiles and checks'] = taipei['the use zoning or compiles and checks'].fillna(0).astype('int64')

# 主要建材數值化

In [27]:
taipei['main building materials'].unique()

array(['鋼筋混凝土造', '見其他登記事項', '鋼骨鋼筋混凝土造', '加強磚造', '鋼骨混凝土造', '見使用執照',
       '預力混凝土造', '磚造', '壁式預鑄鋼筋混凝土造', nan, '木造', '石造', '鋼造', '鋼筋混凝土加強磚造',
       '土造', '混凝土造', '土石造', '鐵造', '竹造', '土磚石混合造'], dtype=object)

In [28]:
len(taipei['main building materials'].unique())

20

In [29]:
mbm = list(taipei['main building materials'].unique())

In [30]:
mname = mbm
mbmname = []
for i in range(1,21):
#     x = i
    mbmname.append(i)

In [31]:
taipei['main building materials'] = taipei['main building materials'].replace(mname,mbmname).astype('int64')

# 鄉鎮市區數值化

In [32]:
taipei['The villages and towns urban district'].unique()

array(['文山區', '大同區', '萬華區', '中正區', '內湖區', '中山區', '南港區', '松山區', '信義區',
       '北投區', '士林區', '大安區', '土城區', '板橋區', '泰山區', '新莊區', '五股區', '林口區',
       '新店區', '金山區', '汐止區', '萬里區', '淡水區', '八里區', '三芝區', '三重區', '蘆洲區',
       '永和區', '中和區', '鶯歌區', '三峽區', '樹林區', '深坑區', '瑞芳區', '石碇區', '石門區',
       '雙溪區', '貢寮區', '坪林區', '平溪區', '烏來區'], dtype=object)

In [33]:
len(taipei['The villages and towns urban district'].unique())

41

In [34]:
ud = list(taipei['The villages and towns urban district'].unique())

In [35]:
dname = ud
udname = []
for i in range(1,42):
#     x = i
    udname.append(i)

In [36]:
taipei['district'] = taipei['The villages and towns urban district'].replace(dname,udname).astype('int64')

# 匯出CSV檔

In [37]:
taipei

,The villages and towns urban district,land sector position building sector house number plate,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,shifting level,total floor number,building state,...,total price NTD,the unit price (NTD / square meter),berth shifting total area square meter,the berth total price NTD,land,building,berth,house age,new addr,district
0,文山區,台北市文山區興隆路四段145巷1~30號,16.40,1,NaN,NaN,2012-10-12,七層,14,1,...,13550000,146147.0,17.45,0,1,1,1,4342 days,台北市文山區興隆路四段145巷15號,1
1,文山區,台北市文山區指南路三段22巷8弄1~30號,52.66,1,NaN,NaN,2012-10-09,六層，電梯樓梯間,7,2,...,17700000,108010.0,0.00,0,1,1,0,5414 days,台北市文山區指南路三段22巷8弄15號,1
2,文山區,台北市文山區興隆路二段233巷5弄1~30號,14.26,1,NaN,NaN,2012-10-20,八層,9,2,...,8010000,128478.0,0.00,0,2,1,1,1067 days,台北市文山區興隆路二段233巷5弄15號,1
3,文山區,台北市文山區羅斯福路五段218巷38弄1~30號,10.68,1,NaN,NaN,2012-10-01,十一層,16,3,...,7800000,173162.0,0.00,0,0,1,0,5526 days,台北市文山區羅斯福路五段218巷38弄15號,1
4,文山區,台北市文山區萬盛街121~150號,30.90,1,NaN,NaN,2012-10-08,五層,7,2,...,22860000,151253.0,10.00,1500000,1,1,1,75 days,台北市文山區萬盛街135號,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466833,土城區,新北市土城區金城路二段121~150號,10.04,1,NaN,NaN,2017-12-05,四層,15,1,...,10300000,99077.0,28.41,0,1,1,1,-833 days,新北市土城區金城路二段135號,13
466834,土城區,新北市土城區國際路158巷1~30號,146.31,0,鄉村區,乙種建築用地,2020-04-02,全,2,9,...,10750000,98443.0,0.00,0,1,1,0,14422 days,新北市土城區國際路158巷15號,13
466835,土城區,新北市土城區金城路二段121~150號,9.78,1,NaN,NaN,2017-12-22,五層,15,1,...,10030000,98401.0,28.41,0,1,1,1,-816 days,新北市土城區金城路二段135號,13
466836,土城區,新北市土城區金城路二段121~150號,9.93,1,NaN,NaN,2018-03-10,十一層,15,1,...,10600000,100445.0,30.90,0,1,1,1,-738 days,新北市土城區金城路二段135號,13


In [38]:
taipei.to_csv("/Users/Howard/Desktop/taipei_clean.csv",index=False, encoding="utf-8")